In [36]:
import pandas as pd
import numpy as np
import pyodbc
from slugify import slugify
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import itertools
import warnings


warnings.simplefilter("ignore")

In [37]:
def get_data(player_id,thresh):
    server = 'localhost\SQLEXPRESS'
    database = 'nba_game_data'

    sql = f'''
    DECLARE @pts_thresh float ;
    SELECT @pts_thresh = {thresh};

    DECLARE @player_id int ;
    SELECT @player_id = {player_id};

    with base as (
    SELECT 
           pgl.[SEASON_YEAR]
          ,pgl.[PLAYER_ID]
          ,pgl.[PLAYER_NAME]
          ,pgl.[TEAM_ID]
          ,pgl.[TEAM_ABBREVIATION]
          ,pgl.oppAbrv
          ,pgl.[TEAM_NAME]
          ,pgl.[GAME_ID]
          ,pgl.yearSeason

          ,pgl.PTS
          ,pgl.[GAME_DATE]
          ,ifp.[PTS_TEAMMATE_ID]
          ,dpm.DID_PLAYER_MISS_GAME as didTeammatePlay

          ,DATEDIFF(day,lag(pgl.GAME_DATE) over(partition by pgl.PLAYER_ID ORDER BY pgl.game_date),pgl.[GAME_DATE])-1 as daysOfRest
          ,case when DATEDIFF(day,lag(pgl.GAME_DATE) over(partition by pgl.PLAYER_ID ORDER BY pgl.game_date),pgl.[GAME_DATE])-1 = 0 then 1 else 0 end as secondHalfOfBackToBack


          ,case when pgl.PTS > @pts_thresh then 1 else 0 end as PtsThresh


          ,lag(pgl.PTS) over(partition by pgl.PLAYER_ID, pgl.oppAbrv ORDER BY pgl.game_date) as PtsPlayerGotLastMeeting

          ,ROUND(SUM(case when pgl.PTS > @pts_thresh then 1 else 0 end) OVER (PARTITION BY pgl.PLAYER_ID,oppAbrv ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) NumberTimesHitAgaisntOppLast2
          ,ROUND(SUM(case when pgl.PTS > @pts_thresh then 1 else 0 end) OVER (PARTITION BY pgl.PLAYER_ID,oppAbrv ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) NumberTimesHitAgaisntOppLast5

          ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID,oppAbrv ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) PtsPerGameAgainstOppLast2
          ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID,oppAbrv ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) PtsPerGameAgainstOppLast5

          ,lag(pgl.PTS) OVER(PARTITION BY pgl.PLAYER_ID ORDER BY pgl.game_date) AS PtsPlayerGotLastGame

          ,ROUND(SUM(CAST(case when pgl.PTS > @pts_thresh then 1 else 0 end AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING),2) 
          /NULLIF(ROUND(count(CAST(case when pgl.PTS > @pts_thresh then 1 else 0 end AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING),2),0) as PctTimesHitLast10
          ,ROUND(SUM(CAST(case when pgl.PTS > @pts_thresh then 1 else 0 end AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) 
          /NULLIF(ROUND(count(CAST(case when pgl.PTS > @pts_thresh then 1 else 0 end AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2),0) as PctTimesHitLast5

          ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PTSLast2
          ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PTSLast8
          ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PTSLast20
          ,ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 82 PRECEDING AND 1 PRECEDING),2) AS PTSPerGameSeason
          ,CASE WHEN 
            ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
            ROUND(AVG(CAST(pgl.PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
          THEN 1 else 0 end as PTSLongTrend


          ,ROUND(AVG(CAST(pgl.MIN AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS MinutesPerGameLast2
          ,ROUND(AVG(CAST(pgl.MIN AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS MinutesPerGameLast8
          ,ROUND(AVG(CAST(pgl.MIN AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS MinutesPerGameLast20

          ,ROUND(SUM(CAST(pgl.FGM AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(pgl.FGA AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalPctLast2
          ,ROUND(SUM(CAST(pgl.FGM AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(pgl.FGA AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalPctLast8
          ,ROUND(SUM(CAST(pgl.FGM AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(pgl.FGA AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalPctLast20

          ,ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalThreePctLast2
          ,ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalThreePctLast8
          ,ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) AS FieldGoalThreePctLast20
          ,CASE WHEN 
            ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),0),2) > 
            ROUND(SUM(CAST(pgl.FG3M AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(pgl.FG3A AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) 
          THEN 1 else 0 end as FieldGoalThreePctTrend

          ,ROUND(AVG(CAST(pgl.PF AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PersonalFoulsLast8
          ,ROUND(AVG(CAST(pgl.PFD AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PersonalFoulsDrawnLast8
          ,CASE WHEN 
            ROUND(AVG(CAST(pgl.PFD AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
            ROUND(AVG(CAST(pgl.PFD AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
          THEN 1 else 0 end as PersonalFoulsDrawnTrend

          ,ROUND(AVG(CAST(pgl.PLUS_MINUS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PlusMinusLast2
          ,ROUND(AVG(CAST(pgl.PLUS_MINUS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PlusMinusLast8
          ,ROUND(AVG(CAST(pgl.PLUS_MINUS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PlusMinusLast20

          ,ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS FantPointsLast2
          ,ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS FantPointsLast8
          ,ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS FantPointsLast20
          ,CASE WHEN 
            ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
            ROUND(AVG(CAST(pgl.NBA_FANTASY_PTS AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
          THEN 1 else 0 end as FantPointsDrawnTrend

          ,ROUND(SUM(CAST(pgl.DD2 AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING),2) AS NumberOfDoubleDoublesLast10
          ,ROUND(SUM(CAST(pgl.TD3 AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID ORDER BY pgl.GAME_DATE ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING),2) AS NumberOfTripleDoublesLast10

          ,RANK() OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ) AS GameNumberThisSeason

          ,ROUND(AVG(CAST(bsa.offensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS OffensiveRatingLast2
          ,ROUND(AVG(CAST(bsa.offensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS OffensiveRatingLast8
          ,ROUND(AVG(CAST(bsa.offensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS OffensiveRatingLast20

          ,ROUND(AVG(CAST(bsa.defensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS DefensiveRatingLast2
          ,ROUND(AVG(CAST(bsa.defensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS DefensiveRatingLast8
          ,ROUND(AVG(CAST(bsa.defensiveRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS DefensiveRatingLast20

          ,ROUND(AVG(CAST(bsa.netRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS NetRatingLast8
          ,ROUND(AVG(CAST(bsa.netRating AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS NetRatingLast20

          ,ROUND(AVG(CAST(bsa.effectiveFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS EffectiveFGPctLast2
          ,ROUND(AVG(CAST(bsa.effectiveFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS EffectiveFGPctLast8
          ,ROUND(AVG(CAST(bsa.effectiveFieldGoalPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS EffectiveFGPctLast20


          ,ROUND(AVG(CAST(bsa.trueShootingPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS TSPctLast2
          ,ROUND(AVG(CAST(bsa.trueShootingPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS TSPctLast8
          ,ROUND(AVG(CAST(bsa.trueShootingPercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS TSPctLast20


          ,ROUND(AVG(CAST(bsa.usagePercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS UsagePctLast2
          ,ROUND(AVG(CAST(bsa.usagePercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS UsagePctLast8
          ,ROUND(AVG(CAST(bsa.usagePercentage AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS UsagePctLast20

          ,ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PossessionsLast2
          ,ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PossessionsLast8
          ,ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PossessionsLast20
          ,CASE WHEN 
            ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
            ROUND(AVG(CAST(bsa.possessions AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
          THEN 1 else 0 end as PossessionsTrend

          ,ROUND(AVG(CAST(bsa.PIE AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PIELast2
          ,ROUND(AVG(CAST(bsa.PIE AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PIELast8
          ,ROUND(AVG(CAST(bsa.PIE AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PIELast20


              ,ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PTSOffTOLast2
          ,ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PTSOffTOLast8
          ,ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PTSOffTOLast20
          ,CASE WHEN 
            ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
            ROUND(AVG(CAST(bsm.pointsOffTurnovers AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
          THEN 1 else 0 end as PTSOffTOTrend

          ,ROUND(AVG(CAST(bsm.pointsSecondChance AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS SecondChancePtsLast2
          ,ROUND(AVG(CAST(bsm.pointsSecondChance AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS SecondChancePtsLast8
          ,ROUND(AVG(CAST(bsm.pointsSecondChance AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS SecondChancePtsLast20

          ,ROUND(AVG(CAST(bsm.pointsFastBreak AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS FastBreakPtsLast2
          ,ROUND(AVG(CAST(bsm.pointsFastBreak AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS FastBreakPtsLast8
          ,ROUND(AVG(CAST(bsm.pointsFastBreak AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS FastBreakPtsLast20

          ,ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),2) AS PaintPtsLast2
          ,ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),2) AS PaintPtsLast8
          ,ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) AS PaintPtsLast20
          ,CASE WHEN 
            ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING),2) > 
            ROUND(AVG(CAST(bsm.pointsPaint AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),2) 
          THEN 1 else 0 end as PaintPtsTrend


          ,ROUND(SUM(CAST(bst.contestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(bst.contestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),0),2) AS ContestedFGPctLast8
          ,ROUND(SUM(CAST(bst.contestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(bst.contestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) AS ContestedFGPctLast20

          ,ROUND(SUM(CAST(bst.uncontestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(bst.uncontestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING),0),2) AS UncontestedFGPctLast2
          ,ROUND(SUM(CAST(bst.uncontestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(bst.uncontestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING),0),2) AS UncontestedFGPctLast8
          ,ROUND(SUM(CAST(bst.uncontestedFieldGoalsMade AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING)/
          NULLIF(SUM(CAST(bst.uncontestedFieldGoalsAttempted AS FLOAT)) OVER (PARTITION BY pgl.PLAYER_ID, pgl.SEASON_YEAR ORDER BY pgl.GAME_DATE ROWS BETWEEN 20 PRECEDING AND 1 PRECEDING),0),2) AS UncontestedFGPctLast20



      FROM [nba_game_data].[dbo].[PlayerGameLogs] pgl

      LEFT OUTER JOIN [nba_game_data].[dbo].[BoxScoreAdvancedV3] bsa
      ON
      cast(pgl.PLAYER_ID as int) = cast(bsa.personId as int)
      AND pgl.GAME_ID = bsa.GAME_ID

       LEFT OUTER JOIN [nba_game_data].[dbo].BoxScoreMiscV3 bsm
       ON
       cast(pgl.PLAYER_ID as int) = cast(bsm.personId as int)
       AND pgl.GAME_ID = bsm.gameId

       LEFT OUTER JOIN [nba_game_data].[dbo].[BoxScorePlayerTrackV3] bst
       ON
       cast(pgl.PLAYER_ID as int) = cast(bst.personId as int)
       AND pgl.GAME_ID = bst.gameId

       LEFT OUTER JOIN [nba_game_data].[dbo].[InfluentialPlayersV1] ifp
       ON
       cast(pgl.PLAYER_ID as int) = cast(ifp.[PLAYER_ID] as int)

       LEFT OUTER JOIN [nba_game_data].[dbo].[GamesPlayerMissedV1] dpm
       ON
       cast(ifp.[PTS_TEAMMATE_ID] as int) = cast(dpm.[PLAYER_ID] as int)
       AND pgl.GAME_ID = dpm.GAME_ID

      WHERE  pgl.PLAYER_ID = @player_id


      )
      select 
      b.PtsThresh,
    
      b.MinutesPerGameLast2,
      b.MinutesPerGameLast8,
      b.MinutesPerGameLast20,
      b.FieldGoalPctLast2,
      b.FieldGoalPctLast8,
      b.FieldGoalPctLast20,
      b.FieldGoalThreePctLast2,
      b.FieldGoalThreePctLast8,
      b.FieldGoalThreePctLast20,
      b.FieldGoalThreePctTrend,
      b.PersonalFoulsLast8,
      b.PersonalFoulsDrawnLast8,
      b.PersonalFoulsDrawnTrend,
      b.PlusMinusLast2,
      b.PlusMinusLast8,
      b.PlusMinusLast20,
      b.FantPointsLast2,
      b.FantPointsLast8,
      b.FantPointsLast20,
      b.FantPointsDrawnTrend,
      b.NumberOfDoubleDoublesLast10,
      b.NumberOfTripleDoublesLast10,
      b.GameNumberThisSeason,
      b.OffensiveRatingLast2,
      b.OffensiveRatingLast8,
      b.OffensiveRatingLast20,
      b.DefensiveRatingLast2,
      b.DefensiveRatingLast8,
      b.DefensiveRatingLast20,
      b.NetRatingLast8,
      b.NetRatingLast20,
      b.EffectiveFGPctLast2,
      b.EffectiveFGPctLast8,
      b.EffectiveFGPctLast20,
      b.TSPctLast2,
      b.TSPctLast8,
      b.TSPctLast20,
      b.UsagePctLast2,
      b.UsagePctLast8,
      b.UsagePctLast20,
      b.PossessionsLast2,
      b.PossessionsLast8,
      b.PossessionsLast20,
      b.PossessionsTrend,
      b.PIELast2,
      b.PIELast8,
      b.PIELast20,
      b.PTSOffTOLast2,
      b.PTSOffTOLast8,
      b.PTSOffTOLast20,
      b.PTSOffTOTrend,
      b.SecondChancePtsLast2,
      b.SecondChancePtsLast8,
      b.SecondChancePtsLast20,
      b.FastBreakPtsLast2,
      b.FastBreakPtsLast8,
      b.FastBreakPtsLast20,
      b.PaintPtsLast2,
      b.PaintPtsLast8,
      b.PaintPtsLast20,
      b.PaintPtsTrend,
      b.ContestedFGPctLast8,
      b.ContestedFGPctLast20,
      b.UncontestedFGPctLast2,
      b.UncontestedFGPctLast8,
      b.UncontestedFGPctLast20,
    ts_allow.[TeamORatingLast2]
      ,ts_allow.[TeamORatingLast10]
      ,ts_allow.[TeamORatingSeason]
      ,ts_allow.[TeamDRatingLast2]
      ,ts_allow.[TeamDRatingLast10]
      ,ts_allow.[TeamDRatingSeason]
      ,ts_allow.[TeamNetRatingLast2]
      ,ts_allow.[TeamNetRatingLast10]
      ,ts_allow.[TeamNetRatingSeason]
      ,ts_allow.[TeamPIELast2]
      ,ts_allow.[TeamPIELast10]
      ,ts_allow.[TeamPIESeason]
      ,ts_allow.[avgPtsAllowedPerGameLast2]
      ,ts_allow.[avgPtsAllowedPerGameLast10]
      ,ts_allow.[avgPtsAllowedPerGameSeason]
      ,ts_allow.[FieldGoalPctAllowedLast2]
      ,ts_allow.[FieldGoalPctAllowedLast10]
      ,ts_allow.[FieldGoalPctAllowedSeason]
      ,ts_allow.[FieldGoal3PctAllowedLast2]
      ,ts_allow.[FieldGoal3PctAllowedLast10]
      ,ts_allow.[FieldGoal3PctAllowedSeason]
      ,ts_allow.[FreeThrowPctAllowedLast2]
      ,ts_allow.[FreeThrowPctAllowedLast10]
      ,ts_allow.[FreeThrowPctAllowedSeason]
      ,ts_allow.[ORatingAllowedLast2]
      ,ts_allow.[ORatingAllowedLast10]
      ,ts_allow.[ORatingAllowedSeason]
      ,ts_allow.[DRatingAllowedLast2]
      ,ts_allow.[DRatingAllowedLast10]
      ,ts_allow.[DRatingAllowedSeason]
      ,ts_allow.[NetRatingAllowedLast2]
      ,ts_allow.[NetRatingAllowedLast10]
      ,ts_allow.[NetRatingAllowedSeason]
      ,ts_allow.[EffFldGoalPctAllowedLast2]
      ,ts_allow.[EffFldGoalPctAllowedLast10]
      ,ts_allow.[EffFldGoalPctAllowedSeason]
      ,ts_allow.[TruShoPctAllowedLast2]
      ,ts_allow.[TruShoPctPctAllowedLast10]
      ,ts_allow.[TruShoPctPctAllowedSeason]
      ,ts_allow.[PaceAllowedLast2]
      ,ts_allow.[PaceAllowedLast10]
      ,ts_allow.[PaceAllowedSeason]
      ,ts_allow.[PacePer40AllowedLast2]
      ,ts_allow.[PacePer40AllowedLast10]
      ,ts_allow.[PacePer40AllowedSeason]
      ,ts_allow.[PossessionsAllowedLast2]
      ,ts_allow.[PossessionsAllowedLast10]
      ,ts_allow.[PossessionsAllowedSeason]
      ,ts_allow.[PIEAllowedLast2]
      ,ts_allow.[PIEAllowedLast10]
      ,ts_allow.[PIEAllowedSeason]
      ,ts_allow.[FGAPct2AtmtAllowedLast2]
      ,ts_allow.[FGAPct2AtmtAllowedLast10]
      ,ts_allow.[FGAPct2AtmtAllowedSeason]
      ,ts_allow.[FGAPct3AtmtAllowedLast2]
      ,ts_allow.[FGAPct3AtmtAllowedLast10]
      ,ts_allow.[FGAPct3AtmtAllowedSeason]
      ,ts_allow.[PctPts2PtrsAllowedLast2]
      ,ts_allow.[PctPts2PtrsAllowedLast10]
      ,ts_allow.[PctPts2PtrsAllowedSeason]
      ,ts_allow.[PctPts3PtrsAllowedLast2]
      ,ts_allow.[PctPts3PtrsAllowedLast10]
      ,ts_allow.[PctPts3PtrsAllowedSeason]
      ,ts_allow.[PctPtsMidRange2PtrsAllowedLast2]
      ,ts_allow.[PctPtsMidRange2PtrsAllowedLast10]
      ,ts_allow.[PctPtsMidRange2PtrsAllowedSeason]
      ,ts_allow.[PctPtsFastBreakPtrsAllowedLast2]
      ,ts_allow.[PctPtsFastBreakPtrsAllowedLast10]
      ,ts_allow.[PctPtsFastBreakPtrsAllowedSeason]
      ,ts_allow.[PctPtsFreeThrwPtrsAllowedLast2]
      ,ts_allow.[PctPtsFreeThrwPtrsAllowedLast10]
      ,ts_allow.[PctPtsFreeThrwPtrsAllowedSeason]
      ,ts_allow.[PctPtsOffTovPtrsAllowedLast2]
      ,ts_allow.[PctPtsOffTovPtrsAllowedLast10]
      ,ts_allow.[PctPtsOffTovPtrsAllowedSeason]
      ,ts_allow.[PctPtsInPaintAllowedLast2]
      ,ts_allow.[PctPtsInPaintAllowedLast10]
      ,ts_allow.[PctPtsInPaintAllowedSeason]
      ,ts_allow.[PctPtsAstsed2AllowedLast2]
      ,ts_allow.[PctPtsAstsed2AllowedLast10]
      ,ts_allow.[PctPtsAstsed2AllowedSeason]
      ,ts_allow.[PctPtsUnAstsed2AllowedLast2]
      ,ts_allow.[PctPtsUnAstsed2AllowedLast10]
      ,ts_allow.[PctPtsUnAstsed2AllowedSeason]
      ,ts_allow.[PctPtsAstsed3AllowedLast2]
      ,ts_allow.[PctPtsAstsed3AllowedLast10]
      ,ts_allow.[PctPtsAstsed3AllowedSeason]
      ,ts_allow.[PctPtsUnAstsed3AllowedLast2]
      ,ts_allow.[PctPtsUnAstsed3AllowedLast10]
      ,ts_allow.[PctPtsUnAstsed3AllowedSeason]
      ,ts_allow.[PctPtsAstsedAllowedLast2]
      ,ts_allow.[PctPtsAstsedAllowedLast10]
      ,ts_allow.[PctPtsAstsedAllowedSeason]
      ,ts_allow.[PctPtsUnAstsedAllowedLast2]
      ,ts_allow.[PctPtsUnAstsedAllowedLast10]
      ,ts_allow.[PctPtsUnAstsedAllowedSeason]
      ,ts_allow.[DistanceAllowedLast2]
      ,ts_allow.[DistanceAllowedLast10]
      ,ts_allow.[DistanceAllowedSeason]
      ,ts_allow.[FTAstAllowedLast2]
      ,ts_allow.[FTAstAllowedLast10]
      ,ts_allow.[FTAstAllowedSeason]
      ,ts_allow.[ContestedFieldGoalPctAllowedLast2]
      ,ts_allow.[ContestedFieldGoalPctAllowedLast10]
      ,ts_allow.[ContestedFieldGoalPctAllowedSeason]
      ,ts_allow.[UnContestedFieldGoalPctAllowedLast2]
      ,ts_allow.[UnContestedFieldGoalPctAllowedLast10]
      ,ts_allow.[UnContestedFieldGoalPctAllowedSeason]
      ,ts_allow.[DefendedAtRimFieldGoalPctAllowedLast2]
      ,ts_allow.[DefendedAtRimFieldGoalPctAllowedLast10]
      ,ts_allow.[DefendedAtRimFieldGoalPctAllowedSeason]

  From base b

  left outer join [nba_game_data].[dbo].[all_team_stats_for_preds] ts_allow
  on 
 b.oppAbrv= ts_allow.TEAM_ABBREVIATION
 and b.GAME_ID = ts_allow.GAME_ID

 where b.yearSeason =2024

  ORDER BY b.GAME_DATE DESC
  '''
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()
    data = pd.read_sql(sql,cnxn)
    return data


In [38]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

sql = f'''
SELECT distinct
[PLAYER_ID]
,[PLAYER_NAME]

FROM [nba_game_data].[dbo].[PlayerGameLogs]
'''
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
players = pd.read_sql(sql,cnxn)

In [56]:
lines

,Player,Line,player_slug,PLAYER_ID,PLAYER_NAME
0,Lauri Markkanen,24.5,lauri-markkanen,1628374,Lauri Markkanen
1,Klay Thompson,11.5,klay-thompson,202691,Klay Thompson
2,Shai Gilgeous-Alexander,32.5,shai-gilgeous-alexander,1628983,Shai Gilgeous-Alexander
3,Bennedict Mathurin,15.5,bennedict-mathurin,1631097,Bennedict Mathurin
4,P.J. Washington,11.5,p-j-washington,1629023,P.J. Washington
...,...,...,...,...,...
93,Deandre Ayton,16.5,deandre-ayton,1629028,Deandre Ayton
94,Jalen Williams,18.5,jalen-williams,1631114,Jalen Williams
95,Nikola Jokic,26.5,nikola-jokic,203999,Nikola Jokic
96,James Harden,17.5,james-harden,201935,James Harden


In [39]:
players['player_slug'] = players['PLAYER_NAME'].apply(lambda x: slugify(x))

In [40]:
lines = pd.read_csv(r'C:\Users\bobby\Downloads\rw-sleeper-predictions-2024-02-25.csv')
# lines

In [41]:
lines['player_slug'] = lines['Player'].apply(lambda x: slugify(x))

In [42]:
lines = pd.merge(lines,players,on=['player_slug'])

In [43]:
best_interactions = {}
for index, row in lines.iterrows():
    try:
            print(row['Player'])
            player_id = row['PLAYER_ID']
            line = row['Line']
            df = get_data(player_id,line)

            df = df.fillna(method='ffill')
            df = df.fillna(method='bfill')

            X = df.drop(['PtsThresh'], axis=1)
            X = X.iloc[1:]
            y = df['PtsThresh']
            y = y.iloc[1:]

        #     # Initialize and train a decision tree model
        #     model = DecisionTreeClassifier(max_depth=3)  # Limit depth for simplicity
        #     model.fit(X, y)

        #     # Plot the decision tree to manually identify interactions (optional)
        #     plt.figure(figsize=(20,10))
        #     plot_tree(model, feature_names=X.columns, filled=True, rounded=True, class_names=str(y.unique()))
        #     plt.title(f"Decision Tree for Player ID {player_id}")
        #     plt.show()

        #     # Automatically identify the top split (best interaction)
        #     # This part simplifies interaction to the most important feature according to the model
        #     best_feature = X.columns[model.tree_.feature[0]]
        #     best_interactions[player_id] = best_feature

             # Generate interaction terms
            poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
            X_poly = poly.fit_transform(X)
            feature_names = poly.get_feature_names_out(X.columns)

            # Train a linear model
            model = LinearRegression()
            model.fit(X_poly, y)

            # Identify the interaction term with the highest coefficient in absolute value
            coeffs = model.coef_
            max_interaction_index = abs(coeffs).argmax()
            best_interaction_feature = feature_names[max_interaction_index]
            best_interactions[player_id] = best_interaction_feature

    except:
        pass
    
print(best_interactions)
    

Lauri Markkanen
Klay Thompson
Shai Gilgeous-Alexander
Bennedict Mathurin
P.J. Washington
Draymond Green
Alex Caruso
De'Aaron Fox
Kevin Durant
Josh Giddey
Aaron Gordon
Andrew Nembhard
Clint Capela
Kyrie Irving
Pascal Siakam
Nick Richards
Jalen Suggs
Buddy Hield
Dejounte Murray
Devin Booker
Kawhi Leonard
Tyus Jones
D'Angelo Russell
Keegan Murray
Luguentz Dort
Coby White
Jordan Clarkson
Austin Reaves
Harrison Barnes
Luka Doncic
Dillon Brooks
Saddiq Bey
Victor Wembanyama
Damian Lillard
Cody Martin
Franz Wagner
Miles Bridges
Paul Reed
Anthony Davis
Ivica Zubac
Eric Gordon
Jonathan Kuminga
Andrew Wiggins
Tre Jones
Brandon Miller
Darius Garland
Kyle Kuzma
Tobias Harris
Jalen Johnson
John Collins
Chet Holmgren
Brandin Podziemski
Jusuf Nurkic
Jamal Murray
Paul George
Myles Turner
Josh Green
Bogdan Bogdanovic
Brook Lopez
Kentavious Caldwell-Pope
Keyonte George
Paolo Banchero
Deni Avdija
Jerami Grant
Rui Hachimura
Terance Mann
Stephen Curry
Fred VanVleet
LeBron James
Giannis Antetokounmpo
Anferne

In [44]:
ds = pd.DataFrame([best_interactions]).T.reset_index()

In [45]:
ds[0].unique()

array(['DefensiveRatingLast2 avgPtsAllowedPerGameLast2',
       'DefensiveRatingLast2 PossessionsLast2',
       'OffensiveRatingLast2 OffensiveRatingLast8',
       'OffensiveRatingLast2 DefensiveRatingLast2',
       'GameNumberThisSeason OffensiveRatingLast2',
       'PlusMinusLast2 OffensiveRatingLast2',
       'TeamORatingLast2 DRatingAllowedLast2',
       'PossessionsLast2 TeamNetRatingLast2',
       'OffensiveRatingLast2 PossessionsLast2',
       'GameNumberThisSeason PossessionsLast2',
       'GameNumberThisSeason TeamNetRatingLast2',
       'TeamDRatingLast2 avgPtsAllowedPerGameLast2',
       'GameNumberThisSeason avgPtsAllowedPerGameLast2',
       'OffensiveRatingLast2 TeamNetRatingLast2',
       'OffensiveRatingLast2 NetRatingLast8',
       'GameNumberThisSeason TeamDRatingLast2',
       'OffensiveRatingLast2 avgPtsAllowedPerGameSeason',
       'FantPointsLast2 OffensiveRatingLast2',
       'TeamNetRatingLast2 NetRatingAllowedLast2',
       'GameNumberThisSeason DefensiveRating

In [46]:
best_interactions

{1628374: 'DefensiveRatingLast2 avgPtsAllowedPerGameLast2',
 202691: 'DefensiveRatingLast2 PossessionsLast2',
 1628983: 'OffensiveRatingLast2 OffensiveRatingLast8',
 1631097: 'OffensiveRatingLast2 DefensiveRatingLast2',
 1629023: 'GameNumberThisSeason OffensiveRatingLast2',
 203110: 'PlusMinusLast2 OffensiveRatingLast2',
 1627936: 'DefensiveRatingLast2 PossessionsLast2',
 1628368: 'TeamORatingLast2 DRatingAllowedLast2',
 201142: 'PlusMinusLast2 OffensiveRatingLast2',
 1630581: 'PossessionsLast2 TeamNetRatingLast2',
 203932: 'DefensiveRatingLast2 PossessionsLast2',
 1629614: 'OffensiveRatingLast2 PossessionsLast2',
 202681: 'OffensiveRatingLast2 PossessionsLast2',
 1627783: 'GameNumberThisSeason PossessionsLast2',
 1630591: 'GameNumberThisSeason TeamNetRatingLast2',
 1627741: 'TeamDRatingLast2 avgPtsAllowedPerGameLast2',
 1627749: 'GameNumberThisSeason avgPtsAllowedPerGameLast2',
 1626164: 'OffensiveRatingLast2 TeamNetRatingLast2',
 202695: 'OffensiveRatingLast2 NetRatingLast8',
 162614

In [ ]:
seasonPtsDataSet

In [53]:
# Define the stored procedure call
stored_proc_call = "{CALL seasonPtsDataSet (?, ?)}"  # Replace with your stored procedure name

# Define parameters for the stored procedure
pts_thresh = 30.5
player_id = 1626157

# Execute the stored procedure
cursor.execute(stored_proc_call, (pts_thresh, player_id))

# Fetch the data
data = cursor.fetchall()

# Execute the stored procedure
cursor.execute(stored_proc_call, (pts_thresh, player_id))

# Fetch the data
data = cursor.fetchall()

# Get column names from cursor description
columns = [column[0] for column in cursor.description]

# Create a DataFrame
df = pd.DataFrame.from_records(data, columns=columns)

# Print or process the DataFrame
df = df.iloc[:-1]  # Example: Print the DataFrame

In [54]:
df

,PtsThresh,didTeammatePlay,daysOfRest,secondHalfOfBackToBack,PtsPlayerGotLastMeeting,NumberTimesHitAgaisntOppLast2,NumberTimesHitAgaisntOppLast5,PtsPerGameAgainstOppLast2,PtsPerGameAgainstOppLast5,PtsPlayerGotLastGame,...,PlusMinusVsFantPts,DefRatingShortVsMedium,MPGRecentVsOffRating,FantPtsVsTeamNetRating,OffRatingVsNetRatingLong,OffRatingMediumVsDefRating,DefRatingVsTeamORating,FantPtsLongVsAvgPtsAllowedRecent,FantPtsVsDefRating,OffRatingVsPTSOffTOTrend
0,0,0,1,0,13,0,1,18.0,21.4,13,...,167.400,10395.0000,2728.1835,-280.3950,1054.4730,11796.4000,11429.0000,4125.920,3069.0000,0.00
1,0,0,0,1,23,0,1,15.5,22.2,24,...,577.600,9918.1845,3909.5030,-243.2000,1433.3005,11092.2615,10900.6800,4366.960,3114.4800,0.00
2,0,0,3,0,17,0,0,12.5,15.8,19,...,418.000,11872.8695,4749.5040,-100.7000,1034.2560,12629.0575,12910.2350,4383.065,4546.7000,0.00
3,0,0,1,0,22,0,0,23.5,22.8,33,...,543.375,10203.6960,4314.1245,-438.7250,777.2490,10832.4000,11310.3000,4730.520,4274.5500,0.00
4,1,0,1,0,27,0,1,27.0,27.6,14,...,243.450,9296.8115,3659.3105,-1.3525,475.9320,9872.0210,11162.1475,4460.785,2606.2675,0.00
5,0,0,1,0,22,0,1,23.5,29.0,19,...,104.850,5468.6375,1665.0000,478.8150,140.0000,5270.0755,3474.8350,4075.920,1906.5225,0.00
6,0,0,1,0,28,0,0,29.0,25.8,29,...,504.900,4503.7200,1933.5590,-45.9000,136.2205,4373.7200,3387.8000,4296.040,2386.8000,0.00
7,0,0,1,0,24,0,0,17.0,19.4,21,...,-124.500,12623.3280,4067.9370,-62.2500,376.0470,12417.4050,15338.7825,4618.950,5148.0750,114.30
8,0,1,1,0,19,0,0,17.5,15.0,19,...,-372.500,13120.7310,3860.8640,122.9250,371.4080,13049.8360,13805.1900,3722.005,4801.5250,111.20
9,0,1,1,0,14,0,0,21.5,22.4,27,...,391.000,9313.2000,4050.1865,-172.0400,524.7525,10331.0850,11109.1750,4374.000,3890.4500,112.85
